# Exercise: Analyzing Chinook Database

Preparation I've done:
 - Retrieve the dataset and load it
 - Load the %sql extension and point it at the database
 - Display the tables and an example query

Additional steps you might take:
 - Add libraries for visualization (matplotlib, seaborn, plotly)
 - Add libraries for statistics (numpy)
 - Explore the dataset using SQL and/or pandas

----

1. Retrieve a list of all the tracks in the database, displaying only the track name and the name of the album it belongs to. Limit the result to the first 5 rows.
   > Operations: `SELECT`
2. Find the total number of customers from each country. Display the country name and the corresponding count. Order the results by the count in descending order.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
3. Identify the top 5 genres with the highest number of tracks. Display the genre name along with the total number of tracks for each genre.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
4. Determine the average invoice total for each customer, considering both the album and individual track purchases. Display the customer's first and last name along with the average invoice total. Order the results by the average invoice total in descending order.
   > Operations: `SELECT`, `AVG`, `JOIN`, `GROUP BY`, `ORDER BY`
5. Identify the customer who spent the most on music purchases. Display the customer's first and last name, along with the total amount spent.
   > Operations: `SELECT`, `SUM`, `JOIN`, `GROUP BY`, `ORDER BY`, `LIMIT`

In [1]:
#run this cell if you have not installed these packages
%pip install pandas sqlalchemy ipython-sql matplotlib seaborn


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load chinook dataset and query it using SQL magic into pandas dataframes
import pandas as pd
import sqlite3
%load_ext sql

# Load data
conn = sqlite3.connect("chinook.sqlite")

# Tell %sql about the database
%sql sqlite:///chinook.sqlite

# List tables in database
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Read data into a Pandas DataFrame
tables = %sql $query

# Print head
display(tables)

# Query to get the first 5 rows of the `albums` table
result = %sql SELECT * FROM albums LIMIT 5;

# Display query result, note that Pandas DataFrame is returned!
display(result)


 * sqlite:///chinook.sqlite
Done.


name
albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists


 * sqlite:///chinook.sqlite
Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
##Q1
list = %sql SELECT  t.Name, a.Title FROM tracks t full join albums a on t.AlbumId = a.AlbumId LIMIT 5;
print(list)

 * sqlite:///chinook.sqlite
Done.
+-----------------------------------------+---------------------------------------+
|                   Name                  |                 Title                 |
+-----------------------------------------+---------------------------------------+
| For Those About To Rock (We Salute You) | For Those About To Rock We Salute You |
|            Balls to the Wall            |           Balls to the Wall           |
|             Fast As a Shark             |           Restless and Wild           |
|            Restless and Wild            |           Restless and Wild           |
|           Princess of the Dawn          |           Restless and Wild           |
+-----------------------------------------+---------------------------------------+


In [5]:
#Q2
query = "SELECT Country, count(CustomerId) count FROM customers group by Country order by count DESC;"
customers = %sql $query
print(customers)

 * sqlite:///chinook.sqlite
Done.
+----------------+-------+
|    Country     | count |
+----------------+-------+
|      USA       |   13  |
|     Canada     |   8   |
|     France     |   5   |
|     Brazil     |   5   |
|    Germany     |   4   |
| United Kingdom |   3   |
|    Portugal    |   2   |
|     India      |   2   |
| Czech Republic |   2   |
|     Sweden     |   1   |
|     Spain      |   1   |
|     Poland     |   1   |
|     Norway     |   1   |
|  Netherlands   |   1   |
|     Italy      |   1   |
|    Ireland     |   1   |
|    Hungary     |   1   |
|    Finland     |   1   |
|    Denmark     |   1   |
|     Chile      |   1   |
|    Belgium     |   1   |
|    Austria     |   1   |
|   Australia    |   1   |
|   Argentina    |   1   |
+----------------+-------+


In [6]:
#Q3
query = "SELECT g.Name, count(t.TrackId) FROM genres g inner join tracks t on g.GenreId= t.GenreId group by g.Name order by count(t.TrackId) desc limit 5;"
genres = %sql $query
print(genres)

 * sqlite:///chinook.sqlite
Done.
+--------------------+------------------+
|        Name        | count(t.TrackId) |
+--------------------+------------------+
|        Rock        |       1297       |
|       Latin        |       579        |
|       Metal        |       374        |
| Alternative & Punk |       332        |
|        Jazz        |       130        |
+--------------------+------------------+


In [7]:
#Q4
query = "SELECT FirstName,LastName, AVG(i.Total) FROM invoices i inner join customers c on i.CustomerId = c.CustomerId group by c.CustomerId order by AVG(i.Total) desc;"
invoice = %sql $query
print(invoice)

 * sqlite:///chinook.sqlite
Done.
+-----------+--------------+--------------------+
| FirstName |   LastName   |    AVG(i.Total)    |
+-----------+--------------+--------------------+
|   Helena  |     Holý     | 7.088571428571429  |
|  Richard  |  Cunningham  | 6.802857142857143  |
|    Luis   |    Rojas     | 6.659999999999999  |
|  Ladislav |    Kovács    | 6.517142857142857  |
|    Hugh   |   O'Reilly   | 6.517142857142857  |
|   Julia   |   Barnett    | 6.231428571428572  |
|   Frank   |   Ralston    | 6.231428571428571  |
|    Fynn   |  Zimmermann  | 6.231428571428571  |
|    Puja   |  Srivastava  | 6.1066666666666665 |
|   Astrid  |    Gruber    | 6.088571428571428  |
|   Victor  |   Stevens    | 6.088571428571428  |
|   Terhi   |  Hämäläinen  | 5.945714285714287  |
| František | Wichterlová  | 5.802857142857143  |
|  Isabelle |   Mercier    | 5.802857142857143  |
|  Johannes | Van der Berg | 5.8028571428571425 |
|    Jack   |    Smith     | 5.660000000000001  |
|    João   |  F

In [8]:
#Q5
query = "SELECT FirstName,LastName, sum(i.Total) FROM invoices i inner join customers c on i.CustomerId = c.CustomerId group by c.CustomerId order by sum(i.Total) desc limit 1;"
purchases = %sql $query
print(purchases)

 * sqlite:///chinook.sqlite
Done.
+-----------+----------+--------------------+
| FirstName | LastName |    sum(i.Total)    |
+-----------+----------+--------------------+
|   Helena  |   Holý   | 49.620000000000005 |
+-----------+----------+--------------------+


In [9]:
t= %sql select * from customers limit 5;
display(t)

 * sqlite:///chinook.sqlite
Done.


CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
